In [1]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator
import re

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [161]:
def rank(text):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    stemmed_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    filtered_words=[]
    for word in stemmed_words:
        filtered=re.sub('[^a-zA-Z]', '', word)
        if(filtered in model.wv.vocab):
            filtered_words.append(word)
    scores=dict()
    counts=dict()
    contextSizes=dict()
    for i in range(len(filtered_words)):
        counts.setdefault(filtered_words[i],0)
        counts[filtered_words[i]]+=1
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        #if(filtered_words[i] in contextSizes or contextSize!=2*window_size+1):
        contextSizes.setdefault(filtered_words[i],contextSize)
        contextSizes[filtered_words[i]]+=contextSize
        for j in range(i+1,rightBound+1):
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            similarity_score=(similarity_score+1)/2.0
            scores.setdefault(filtered_words[i],similarity_score)
            scores.setdefault(filtered_words[j],similarity_score)
            scores[filtered_words[i]]+=similarity_score#min(scores[filtered_words[i]],similarity_score)
            scores[filtered_words[j]]+=similarity_score#min(scores[filtered_words[j]],similarity_score)
    wordScores=list(map(list, scores.items()))
    for i in range(len(wordScores)):
        contextSize=contextSizes[wordScores[i][0]] #if wordScores[i][0] in contextSizes else 2*window_size-1 
        wordScores[i][1]=wordScores[i][1]/(contextSize)
    wordScores=sorted(wordScores, key = lambda x: x[1])#,reverse=True)
    filteredWordScores=[]
    for word in wordScores:
        filtered=re.sub('[^a-zA-Z]', '', word[0])
        if(len(filtered)>2):
            filteredWordScores.append([filtered,word[1]])
    return filteredWordScores

In [160]:
rank("""

Sgt. 1st Class Michael James Goble died due to injuries resulting from an explosion on Dec. 22 in the Taliban-contested northern Afghan province of Kunduz. Two Pentagon officials tell NPR Goble's unit was seizing a Taliban weapons cache when he was fatally wounded.

It was Goble's third tour of duty in Afghanistan, and he died three weeks before what would have been his 34th birthday.

The Westwood, N.J., native is survived by his partner and their young daughter.

Goble enlisted in the U.S. Army when he was 18 as a Special Forces candidate and earned a Green Beret three years later. He was most recently assigned to the 1st Battalion, 7th Special Forces Group (Airborne) at Eglin Air Force Base in Florida.

"Sgt. 1st Class Goble was more than just a member of the 7th Special Forces Group," the group's commander, Col. John Sannes, said in a statement from the U.S. Army Special Operations Command. "He was a brother to us and a beloved family member to the Northwest Florida community."

"We will honor our brother's sacrifice and provide the best possible care to his family," Sannes added.

The circumstances of Goble's death were disputed in a tweet by Taliban spokesman Zabidullah Mujahid. He claimed his forces blew up an American vehicle in Kunduz province on Sunday with an improvised explosive device, killing one American and wounding another along with an Afghan commando.

Districts in Kunduz province, including its eponymous capital, have come under repeated attacks and been briefly occupied by Taliban forces over the past four years. U.S. aircraft mistakenly bombed a trauma center run by Doctors Without Borders in the city of Kunduz in 2015, killing scores of patients and hospital workers.

More than 2,400 American forces have been killed in Afghanistan since the U.S. invaded that nation and overthrew its Taliban-controlled government, which continued harboring al-Qaida leaders after the Sept. 11, 2001 attacks in the U.S.

The U.S. resumed peace negotiations with the Taliban in Qatar on Dec. 7. Those talks had been halted by President Trump in a September tweet after a suicide bombing in Kabul killed a U.S. soldier.

The U.S. is seeking a firm commitment from the Taliban not to allow Afghanistan to continue as a base of operations for al-Qaida and Islamic State insurgents.

The Taliban, which has refused to negotiate directly with an Afghan government it considers illegitimate, is demanding that Washington remove all of the approximately 13,000 American forces still fighting in what has become the longest war in U.S. history.

More than eighteen years after being expelled from power, the Taliban continue to dominate in many parts of Afghanistan. More than half of Afghanistan's population is located in districts either controlled or contested by the Taliban.
""")

[['taliban', 0.054222761358444886],
 ['force', 0.05639106034308613],
 ['afghanistan', 0.08826384606904217],
 ['american', 0.0998080101828756],
 ['special', 0.10209932577397142],
 ['afghan', 0.12557344352050373],
 ['province', 0.13030662578308866],
 ['year', 0.13304821279650372],
 ['group', 0.13841838562317813],
 ['dec', 0.17050124707055234],
 ['tweet', 0.17125326727649995],
 ['florida', 0.17442986648529768],
 ['died', 0.17627185037625687],
 ['attack', 0.17657466596435933],
 ['base', 0.17789032873475835],
 ['operation', 0.17804899326126492],
 ['class', 0.17842133453507958],
 ['member', 0.18051825956042325],
 ['brother', 0.1808871851480078],
 ['killing', 0.1811522598422709],
 ['district', 0.18119455647787877],
 ['continue', 0.18334001741771186],
 ['three', 0.18487973371520638],
 ['killed', 0.1878426314464637],
 ['army', 0.18870439646499498],
 ['family', 0.18927055384431565],
 ['government', 0.19039210286878405],
 ['sgt', 0.19652114994823933],
 ['score', 0.26795570991401163],
 ['col', 0.2